In [1]:
import redis

In [2]:
r = redis.Redis(
    host='redis-18884.c280.us-central1-2.gce.cloud.redislabs.com',
    port=18884,
    password='BtBJ4VfuHryIYpZrfVqaxWPsRkrKzukf'
)

In [5]:
d = {
  #b"attachments": [],
  "tts": False,
  #b"embeds": [],
  "timestamp": "2017-07-11T17:27:07.299000+00:00",
  "mention_everyone": False,
  "id": 334385199974967042,
  "pinned": False,
  "edited_timestamp": None,
  "author": 53908099506183680,
  #b"mention_roles": [],
  "content": "Supa Hot",
  "channel_id": 290926798999357250,
  #b"mentions": [],
  "type": 0
}

In [47]:
from datetime import datetime

class Message:
    __slots__ = ('tts', 'timestamp', 'me', 'id', 'pinned', 'edited_timestamp', 'author', 'content', 'channel_id', 'type')

    def __init__(self, data):
        self.tts = data['tts']
        self.timestamp = datetime.fromisoformat(data['timestamp'])
        self.me = data['mention_everyone']
        self.id = data['id']
        self.pinned = data['pinned']
        self.edited_timestamp = data['edited_timestamp']
        self.author = data['author']
        self.content = data['content']
        self.channel_id = data['channel_id']
        self.type = data['type']

In [2]:
%load_ext cython

In [7]:
%%cython
from datetime import datetime

cdef class CMessage:
    cdef bint tts
    cdef object timestamp
    cdef bint mention_everyone
    cdef unsigned long long id
    cdef bint pinned
    cdef object edited_timestamp
    cdef unsigned long long author
    cdef str content
    cdef unsigned long long channel_id
    cdef char type

    def __init__(self, dict data):
        self.tts = data['tts']
        self.timestamp = datetime.fromisoformat(data['timestamp'])
        self.mention_everyone = data['mention_everyone']
        self.id = data['id']
        self.pinned = data['pinned']
        self.edited_timestamp = data['edited_timestamp']
        self.author = data['author']
        self.content = data['content']
        self.channel_id = data['channel_id']
        self.type = data['type']

In [8]:
msg = CMessage(d)    

In [9]:
from pympler import asizeof

asizeof.asizesof(msg)

(112,)

In [25]:
import json
import msgpack

r.delete('messages/290926798999357250-334385199974967042')
r.set('messages/290926798999357250-334385199974967042', json.dumps(d).encode())

True

In [26]:
# hashes: 294
# JSON: 352
# MSGPack: 320

r.memory_usage('messages/290926798999357250-334385199974967042')

352

In [19]:
import threading
res = 'NOTSET'

def func():
    global res
    res = r.blpop('new')

t = threading.Thread(target=func)
t.start()

In [23]:
res

'NOTSET'

In [13]:
t.join()

In [29]:
r.get('new')

ResponseError: WRONGTYPE Operation against a key holding the wrong kind of value